# Installation of Necessary Libraries

In [20]:
#requirements.txt in the Same Directory
!pip install -r requirements.txt

  Using cached certifi-2021.5.30-py2.py3-none-any.whl (145 kB)
  Using cached charset_normalizer-2.0.6-py3-none-any.whl (37 kB)
  Using cached idna-3.3-py3-none-any.whl (61 kB)
  Using cached language_tool_python-2.6.0-py3-none-any.whl (30 kB)
  Using cached nltk-3.6.3-py3-none-any.whl (1.5 MB)
  Using cached pandas-1.3.3.tar.gz (4.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  pip subprocess to install build dependencies did not run successfully.
  exit code: 1
  
  [322 lines of output]
  Ignoring numpy: markers 'python_version == "3.7" and (platform_machine != "arm64" or platform_system != "Darwin") and platform_machine != "aarch64"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.8" and (platform_machine != "arm64" or platform_system != "Darwin") and platform_machine != "aarch64"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.7" and platform_machine == "aarch64"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.8" and platform_machine == "aarch64"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.8" and platform_machine == "arm64" and platform_system == "Darwin"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.9" and platform_machine == "arm64" and platform_system == "Da

In [2]:
# #requirements.txt NOT in the same Directory
#!pip install -r /path/to/requirements.txt

# Importing Libraries and Setting Up

In [3]:
import re
import pandas as pd
import language_tool_python
from language_tool_python import LanguageTool
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, pipeline
import torch
import csv
from textblob import TextBlob
import random
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

From C:\Users\SNEHA\anaconda3\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.



In [4]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SNEHA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SNEHA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SNEHA\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Data Loading

Loads the text data from a specified file path. This data consists of customer queries and engineer responses that will be used for analysis.

In [5]:
#put path where you have Gen ai __ case study:
file_path = r'D:\Downloads\Gen ai __ case study.txt'
    
with open(file_path, 'r') as file:
    content = file.read()

content[:]

"ï»¿Ticket Id - 34333\nDescription - Integration request\nCustomer Response:\n\n\nHi Support,\n\n\nThere has been a mistake, you integrated a Store located in Mexico, the one we were trying to integrate is in Madrid, please delete and cancel these:\n\n\n\n\n7dad6eeb-4 \n\n\nLet us know.\n\n\nThanks.\n\n\nEngineer Response:\n\n\nHi Roberto,\n\n\nWe have de-staged the stores with lastpos and provided menu maker access.\n\n\nStore UUID :\n7dad6eeb-4746  \n\tA46d97e2-e99 \n\tA26f3b8a-4ffc 02\n\tEfb224ca-04 \n\t\n\nIf you feel your request was not addressed/resolved properly, you can reopen the same case within 3 business working days and/or request escalation. If you have a new issue, please raise a new case.\n\n\nRegards,\nL2 UET Support\n\n\n\n\n\n\n\n\nEmpathy - Fail\n\n\nComment -   Customer has stated that the previous eng has integrated the wrong store as address is different. Current Eng should have shown Empathy to customer since previous Eng handled it incorrectly without taking c

# Extracting Tickets and Responses

Defines and applies a function to extract individual tickets and their respective customer and engineer responses from the loaded data.

In [6]:
def extract_tickets_and_responses(content):
    tickets = []
    ticket_blocks = content.split('Ticket Id - ')[1:]  

    for block in ticket_blocks:
        
        ticket_id_search = re.search(r'(\d+)', block)
        ticket_id = ticket_id_search.group(1) if ticket_id_search else "Unknown"

        
        customer_response_search = re.search(r'Customer Response:\s*(.*?)(?:Engineer Response:|$)', block, re.DOTALL)
        engineer_response_search = re.search(r'Engineer Response:\s*(.*?)(?:Empathy -|$)', block, re.DOTALL)
                                            

        customer_response = customer_response_search.group(1).strip() if customer_response_search else ""
        engineer_response = engineer_response_search.group(1).strip() if engineer_response_search else ""

        tickets.append({
            'ticket_id': ticket_id,
            'customer_response': customer_response,
            'engineer_response': engineer_response
        })

    return tickets

data = extract_tickets_and_responses(content)
print(data)


[{'ticket_id': '34333', 'customer_response': 'Hi Support,\n\n\nThere has been a mistake, you integrated a Store located in Mexico, the one we were trying to integrate is in Madrid, please delete and cancel these:\n\n\n\n\n7dad6eeb-4 \n\n\nLet us know.\n\n\nThanks.', 'engineer_response': 'Hi Roberto,\n\n\nWe have de-staged the stores with lastpos and provided menu maker access.\n\n\nStore UUID :\n7dad6eeb-4746  \n\tA46d97e2-e99 \n\tA26f3b8a-4ffc 02\n\tEfb224ca-04 \n\t\n\nIf you feel your request was not addressed/resolved properly, you can reopen the same case within 3 business working days and/or request escalation. If you have a new issue, please raise a new case.\n\n\nRegards,\nL2 UET Support'}, {'ticket_id': '338', 'customer_response': "Hi guys\n\n\nThe request wasn't about access to menu maker, the request was the get help from support in building out and applying modifiers?\n\n\nThanks\nStefan", 'engineer_response': 'Hi Stefan,\n\n\nPlease follow the below documents to update the 

#  Data Preprocessing

Cleans and preprocesses the extracted tickets. This includes removing newline characters and additional whitespace, ensuring the text data is in a suitable format for grammer analysis.

*not applying tokenization,stop words and stemming/lemmatizer because need to maintain the raw data for grammar analysis.

In [7]:
def clean_text(text):
    """
    Function to clean text by removing newline characters, equals signs,
    and performing optional preprocessing.
    """
    cleaned_text = re.sub(r'\s+', ' ', text).strip()

    return cleaned_text

def preprocess_tickets(tickets):
    """
    Function to preprocess a list of tickets, cleaning text and handling missing data.
    """
    cleaned_tickets = []
    for ticket in tickets:
        
        if not ticket['customer_response'].strip() and not ticket['engineer_response'].strip():
            continue

        # Clean text
        ticket['customer_response'] = clean_text(ticket['customer_response'])
        ticket['engineer_response'] = clean_text(ticket['engineer_response'])

        cleaned_tickets.append(ticket)

    return cleaned_tickets

cleaned_data = preprocess_tickets(data)
print(cleaned_data)

[{'ticket_id': '34333', 'customer_response': 'Hi Support, There has been a mistake, you integrated a Store located in Mexico, the one we were trying to integrate is in Madrid, please delete and cancel these: 7dad6eeb-4 Let us know. Thanks.', 'engineer_response': 'Hi Roberto, We have de-staged the stores with lastpos and provided menu maker access. Store UUID : 7dad6eeb-4746 A46d97e2-e99 A26f3b8a-4ffc 02 Efb224ca-04 If you feel your request was not addressed/resolved properly, you can reopen the same case within 3 business working days and/or request escalation. If you have a new issue, please raise a new case. Regards, L2 UET Support'}, {'ticket_id': '338', 'customer_response': "Hi guys The request wasn't about access to menu maker, the request was the get help from support in building out and applying modifiers? Thanks Stefan", 'engineer_response': 'Hi Stefan, Please follow the below documents to update the modifiers in the menu payload. https://develo https:/develo Store UUID: 7140fa

# Grammar Checking Function

Implements a function to check the grammar of the engineer responses using `language_tool_python`. It scores the responses based on the number of grammatical errors and flags responses with significant errors.

In [8]:
def check_grammar(text, max_threshold=10):
    tool = language_tool_python.LanguageTool('en-US')
    text = " ".join("<LINK>" if token.startswith("http") or token.startswith("www") or token.startswith("https")  else token for token in text.split())
    matches = tool.check(text)
    num_errors = len(matches)

    if num_errors >= max_threshold:
        score = 0
    else:
        # Linear scaling of the score
        score = round(10 * (1 - num_errors / max_threshold)**2)

    if score >= 8:
        print({"score": score, "status": "Pass", "comment": "No grammatical errors found."},"\n")
        return {"score": score, "status": "Pass", "comment": "No grammatical errors found."}
    else:
        errors = [f"{match.message} (Suggested word: {match.replacements[0]})" for match in matches]
        print({"score": score, "status": "Fail", "comment": "; ".join(errors)},"\n")
        return {"score": score, "status": "Fail", "comment": "; ".join(errors)}
result_data = []
for i in cleaned_data:
    print("SENTENCE: ",i['engineer_response'],"\n")
    grammar_result = check_grammar(i['engineer_response'])
    result_data.append({"ticket_id": i['ticket_id'], "parameter": "grammar", **grammar_result})

SENTENCE:  Hi Roberto, We have de-staged the stores with lastpos and provided menu maker access. Store UUID : 7dad6eeb-4746 A46d97e2-e99 A26f3b8a-4ffc 02 Efb224ca-04 If you feel your request was not addressed/resolved properly, you can reopen the same case within 3 business working days and/or request escalation. If you have a new issue, please raise a new case. Regards, L2 UET Support 

{'score': 6, 'status': 'Fail', 'comment': 'Possible spelling mistake found. (Suggested word: restaged); Possible spelling mistake found. (Suggested word: lasts)'} 

SENTENCE:  Hi Stefan, Please follow the below documents to update the modifiers in the menu payload. https://develo https:/develo Store UUID: 7140faa If you feel the issue is not addressed/resolved properly you can reopen the same case within 3 business working days or If you have a new issue please raise a new case. Thanks, L2 UET Support 

{'score': 10, 'status': 'Pass', 'comment': 'No grammatical errors found.'} 

SENTENCE:  Hi Pankaj, P

# Text Preprocessing for NLP

Further preprocesses the text data by tokenizing, removing stopwords, and lemmatizing the words. This step is crucial for effective NLP analysis. 

In [9]:
lemmatizer = WordNetLemmatizer()

def process_text(text):

    words = word_tokenize(text)

    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words]

    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
    processed_text = ' '.join(lemmatized_words)

    return processed_text

for entry in cleaned_data:
    entry['processed_customer_response'] = process_text(entry['customer_response'])
    entry['processed_engineer_response'] = process_text(entry['engineer_response'])
print(cleaned_data)

original_data = pd.DataFrame(cleaned_data)

[{'ticket_id': '34333', 'customer_response': 'Hi Support, There has been a mistake, you integrated a Store located in Mexico, the one we were trying to integrate is in Madrid, please delete and cancel these: 7dad6eeb-4 Let us know. Thanks.', 'engineer_response': 'Hi Roberto, We have de-staged the stores with lastpos and provided menu maker access. Store UUID : 7dad6eeb-4746 A46d97e2-e99 A26f3b8a-4ffc 02 Efb224ca-04 If you feel your request was not addressed/resolved properly, you can reopen the same case within 3 business working days and/or request escalation. If you have a new issue, please raise a new case. Regards, L2 UET Support', 'processed_customer_response': 'Hi Support , mistake , integrated Store located Mexico , one trying integrate Madrid , please delete cancel : 7dad6eeb-4 Let u know . Thanks .', 'processed_engineer_response': 'Hi Roberto , de-staged store lastpos provided menu maker access . Store UUID : 7dad6eeb-4746 A46d97e2-e99 A26f3b8a-4ffc 02 Efb224ca-04 feel request

# Empathy Scoring Using TextBlob

Utilizes TextBlob to calculate empathy scores for the processed engineer responses. Scores are determined based on sentiment polarity and scaled to a range of 0-10.

In [10]:
def calculate_empathy_scores(sentence):
    blob = TextBlob(sentence)
    empathy_score = round((blob.sentiment.polarity + 1) * 5) #Range: -1 (negative) to 1 (positive)
    return empathy_score

for i in cleaned_data:
    empathy_score = calculate_empathy_scores(i['processed_engineer_response'])
    print(f"Sentence: {i['engineer_response']}\nEmpathy Score: {empathy_score}\n")

Sentence: Hi Roberto, We have de-staged the stores with lastpos and provided menu maker access. Store UUID : 7dad6eeb-4746 A46d97e2-e99 A26f3b8a-4ffc 02 Efb224ca-04 If you feel your request was not addressed/resolved properly, you can reopen the same case within 3 business working days and/or request escalation. If you have a new issue, please raise a new case. Regards, L2 UET Support
Empathy Score: 5

Sentence: Hi Stefan, Please follow the below documents to update the modifiers in the menu payload. https://develo https:/develo Store UUID: 7140faa If you feel the issue is not addressed/resolved properly you can reopen the same case within 3 business working days or If you have a new issue please raise a new case. Thanks, L2 UET Support
Empathy Score: 6

Sentence: Hi Pankaj, Please try understanding the situation. All the menu changes need to be done from POS side when the store was POS integrated. We understand that the uber has 2 pricing options but that also needs to be updated from

#  Text Similarity Analysis

Computes the cosine similarity between customer queries and engineer responses to assess the relevance of responses to the queries.

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

for _, row in original_data.iterrows():

    customer_response = row['processed_customer_response']
    engineer_response = row['processed_engineer_response']

    vectorizer = CountVectorizer().fit_transform([customer_response, engineer_response])
    vectors = vectorizer.toarray()

    # Calculate cosine similarity
    cosine_sim = cosine_similarity(vectors)[0, 1]

    similarity_threshold = 0.5
    print("Engineer Response:", row['engineer_response'],"\n")
    # Check if engineer's response is similar enough to customer's query
    if cosine_sim >= similarity_threshold:
        print("Relevance: Engineer's response is relevant to the customer's query.\n")
    else:
        print("Relevance: Engineer's response may not be highly relevant to the customer's query.\n")


Engineer Response: Hi Roberto, We have de-staged the stores with lastpos and provided menu maker access. Store UUID : 7dad6eeb-4746 A46d97e2-e99 A26f3b8a-4ffc 02 Efb224ca-04 If you feel your request was not addressed/resolved properly, you can reopen the same case within 3 business working days and/or request escalation. If you have a new issue, please raise a new case. Regards, L2 UET Support 

Relevance: Engineer's response may not be highly relevant to the customer's query.

Engineer Response: Hi Stefan, Please follow the below documents to update the modifiers in the menu payload. https://develo https:/develo Store UUID: 7140faa If you feel the issue is not addressed/resolved properly you can reopen the same case within 3 business working days or If you have a new issue please raise a new case. Thanks, L2 UET Support 

Relevance: Engineer's response may not be highly relevant to the customer's query.

Engineer Response: Hi Pankaj, Please try understanding the situation. All the men

#  Additional Empathy Analysis with DistilBERT

Performs empathy analysis using the DistilBERT model. This cell demonstrates an alternative approach using a different pre-trained model for empathy scoring.

In [12]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

def calculate_empathy_scores_with_distilbert(engineer_responses):
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

    empathy_scores = []
    for response in engineer_responses:
        inputs = tokenizer(response, return_tensors='pt', truncation=True, padding=True)
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
        sentiment_score = probabilities[0][1].item()  # Probability of positive sentiment
        sentiment_score = round(sentiment_score * 10)
        empathy_threshold = 6
        
        is_empathetic = sentiment_score > empathy_threshold

        empathy_scores.append((response, is_empathetic, sentiment_score))

    return empathy_scores

engineer_responses = [i['engineer_response'] for i in cleaned_data]
empathy_scores = calculate_empathy_scores_with_distilbert(engineer_responses)


for response, is_empathetic, sentiment_score in empathy_scores:
    print(f"Engineer Response: {response}\n")
    print(f"Is Empathetic: {is_empathetic}")
    print(f"Sentiment Score: {sentiment_score}")
    print("Status: FAIL","Comment: Engineer needs to be more empathetic\n")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Engineer Response: Hi Roberto, We have de-staged the stores with lastpos and provided menu maker access. Store UUID : 7dad6eeb-4746 A46d97e2-e99 A26f3b8a-4ffc 02 Efb224ca-04 If you feel your request was not addressed/resolved properly, you can reopen the same case within 3 business working days and/or request escalation. If you have a new issue, please raise a new case. Regards, L2 UET Support

Is Empathetic: False
Sentiment Score: 5
Status: FAIL Comment: Engineer needs to be more empathetic

Engineer Response: Hi Stefan, Please follow the below documents to update the modifiers in the menu payload. https://develo https:/develo Store UUID: 7140faa If you feel the issue is not addressed/resolved properly you can reopen the same case within 3 business working days or If you have a new issue please raise a new case. Thanks, L2 UET Support

Is Empathetic: False
Sentiment Score: 5
Status: FAIL Comment: Engineer needs to be more empathetic

Engineer Response: Hi Pankaj, Please try understand

#  Sentiment Analysis with Hugging Face Pipeline

Sets up a sentiment analysis pipeline using Hugging Face's `transformers` and applies it to analyze the empathy in engineer responses. Converts sentiment scores to a 1-10 scale for empathy

In [13]:
# Initialize the sentiment analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis")

def analyze_empathy(response):
    """
    Analyze the empathy in the engineer response using a sentiment analysis model.
    Convert the sentiment score to a 1-10 scale for empathy.
    """
    results = sentiment_analyzer(response)
    
    if results[0]['label'] == 'NEGATIVE':
        empathy_score = round((results[0]['score']) * 5)
    else:
        empathy_score = round(results[0]['score'] * 5)
    empathy_comment = "Highly empathetic" if empathy_score > 7 else "Moderately empathetic" if empathy_score > 4 else "Low empathy"
    return empathy_score, empathy_comment
for _, row in original_data.iterrows():
    print(analyze_empathy(row['engineer_response']),"\n")
    print(row['engineer_response'],"\n")
    

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


(5, 'Moderately empathetic') 

Hi Roberto, We have de-staged the stores with lastpos and provided menu maker access. Store UUID : 7dad6eeb-4746 A46d97e2-e99 A26f3b8a-4ffc 02 Efb224ca-04 If you feel your request was not addressed/resolved properly, you can reopen the same case within 3 business working days and/or request escalation. If you have a new issue, please raise a new case. Regards, L2 UET Support 

(5, 'Moderately empathetic') 

Hi Stefan, Please follow the below documents to update the modifiers in the menu payload. https://develo https:/develo Store UUID: 7140faa If you feel the issue is not addressed/resolved properly you can reopen the same case within 3 business working days or If you have a new issue please raise a new case. Thanks, L2 UET Support 

(5, 'Moderately empathetic') 

Hi Pankaj, Please try understanding the situation. All the menu changes need to be done from POS side when the store was POS integrated. We understand that the uber has 2 pricing options but tha

# Generating labeled data

We are manually creating labeled data by using a function that is designed to assess the empathetic nature of an engineer's response by analyzing the sentiment of the given text. We are giving sample engineer and customer responses for training our models.

In [14]:
# Function to generate empathy label based on sentiment analysis
def generate_empathy_label(engineer_response):
    blob = TextBlob(engineer_response)
    sentiment_score = blob.sentiment.polarity  # Range: -1 (negative) to 1 (positive)
    
    empathy_label = round((sentiment_score + 1) * 5.5)
    
    return max(0, min(10, empathy_label)) 

engineer_responses = [
    "I understand your frustration. Let's work on a solution.",
    "I apologize for the poor service. We'll investigate and make improvements.",
    "You're welcome! I'm glad I could assist you.",
    "I'm sorry for the inconvenience. Let me see what I can do to help.",
    "I'm sorry to hear that. Can you provide more details so I can better assist you?",
    "I'm sorry to hear that. We value your feedback and will strive to improve.",
    "You're welcome! If you have any more questions, feel free to ask.",
    "I'm sorry to hear about your experience. We'll investigate and address the issue.",
    "I understand your frustration. Let's work together to find a solution.",
    "I apologize if my assistance was not satisfactory. Please let me know how I can help.",
    "I'm glad we could resolve the issue to your satisfaction.",
    "Thank you for bringing this matter to our attention. We'll address it promptly.",
    "Your feedback is crucial for our continuous improvement. We value your input.",
    "I apologize for any inconvenience caused. Let's work together to find a solution.",
    "I understand the frustration this may have caused. We're here to assist you.",
    "We appreciate your loyalty to our services. Your satisfaction is important to us.",
    "I'm committed to ensuring a positive experience for you. Let me know how I can help.",
    "Your concerns are heard, and we're taking steps to prevent a recurrence.",
    "I'm sorry if our service did not meet your expectations. We'll learn from this.",
    "I appreciate your understanding. We'll make improvements to avoid similar issues.",
    "I appreciate your patience as we work to resolve this issue.",
    "I'm committed to ensuring your satisfaction. Let's address this concern together.",
    "Your insights are valuable to us. We'll use them to enhance our services.",
    "I apologize for any inconvenience caused. We'll take corrective measures.",
    "Thank you for bringing this to our attention. We'll investigate and respond promptly.",
    "I'm here to assist you with any questions or issues you may have.",
    "Your feedback is crucial in helping us improve. We'll learn from this experience.",
    "I understand the challenges you're facing. We'll explore solutions to make it right.",
    "I'm sorry if our service fell short of your expectations. We're committed to improvement.",
    "Your satisfaction is our priority. We'll take immediate action to address your concerns.",
    "I'm glad we could resolve your issue. If you have more questions, feel free to ask.",
    "I understand the importance of your concern. We'll prioritize this matter.",
    "I apologize for any inconvenience. Let's work together to find a solution.",
    "Thank you for reaching out. We'll investigate and get back to you shortly.",
    "I appreciate your detailed explanation. It helps us understand the situation better.",
    "I'm here to assist you at any time. Your satisfaction is our goal.",
    "Your feedback drives our improvement. We'll take your suggestions into account.",
    "I'm sorry to hear about your experience. We'll make sure this doesn't happen again.",
    "Your positive feedback motivates us to provide excellent service. Thank you!",
    "I understand this has been frustrating. We'll do our best to make it right.",
    "I appreciate your patience. We're working on resolving the issue.",
    "I'm thrilled to hear that you're satisfied with our service!",
    "I'm sorry for any confusion. Let me clarify the steps for you.",
    "Your feedback is valuable. We'll use it to enhance our services.",
    "Thank you for bringing this to our attention. We'll investigate immediately.",
    "I'm here to help. Let me know if there's anything else you need assistance with.",
    "I understand this is frustrating. We'll expedite the resolution process.",
    "Your positive experience is our priority. Let us know if there's anything we can improve.",
    "I'm sorry to hear about the inconvenience. We'll take immediate action to address it.",
    "Your satisfaction is important to us. We'll do our best to make things right."
]

customer_complaints = [
    "There has been a mistake in your service. I'm not satisfied.",
    "I'm not happy with your response. It wasn't helpful.",
    "Your service is terrible. I'm disappointed.",
    "I'm frustrated with the poor service. Can you fix it?",
    "Your assistance didn't address my concerns. I'm unhappy.",
    "I have a problem with your service. Can you help?",
    "I'm dissatisfied with the resolution. It's unacceptable.",
    "I'm not pleased with your support. It's frustrating.",
    "Your service is not helpful. I'm disappointed.",
    "I'm having issues with your service. Can you resolve them?",
    "I'm not satisfied with your assistance. It's frustrating.",
    "Your support is not up to the mark. I'm disappointed.",
    "I'm not happy with your service. It's unacceptable.",
    "I'm disappointed with your response. It's not satisfactory.",
    "Your service fell short of my expectations. I'm frustrated.",
    "I'm not happy with the outcome. It's disappointing.",
    "Your service is causing inconvenience. Can you fix it?",
    "I'm dissatisfied with your resolution. It's not helpful.",
    "I'm facing problems with your service. Can you assist?",
    "I'm unhappy with your assistance. It's frustrating.",
    "Your support is not satisfactory. I'm not pleased.",
    "I'm frustrated with the inconvenience caused by your service.",
    "I'm disappointed with your resolution. It's not acceptable.",
    "Your service is causing frustration. Can you address it?",
    "I'm dissatisfied with your support. It's not helpful.",
    "I'm not pleased with your assistance. It's frustrating.",
    "Your service is not satisfactory. I'm disappointed.",
    "I'm frustrated with your response. It's not helpful.",
    "I'm not happy with your support. It's disappointing.",
    "Your resolution is not satisfactory. I'm disappointed.",
    "I'm not pleased with your assistance. It's frustrating.",
    "Your service is not helpful. I'm disappointed.",
    "I'm disappointed with your response. It's not satisfactory.",
    "Your service fell short of my expectations. I'm frustrated.",
    "I'm not happy with the outcome. It's disappointing.",
    "Your service is causing inconvenience. Can you fix it?",
    "I'm dissatisfied with your resolution. It's not helpful.",
    "I'm facing problems with your service. Can you assist?",
    "I'm unhappy with your assistance. It's frustrating.",
    "Your support is not satisfactory. I'm not pleased.",
    "I'm frustrated with the inconvenience caused by your service.",
    "I'm disappointed with your resolution. It's not acceptable.",
    "Your service is causing frustration. Can you address it?",
    "I'm dissatisfied with your support. It's not helpful.",
    "I'm not pleased with your assistance. It's frustrating.",
    "Your service is not satisfactory. I'm disappointed.",
    "I'm frustrated with your response. It's not helpful.",
    "I'm not happy with your support. It's disappointing.",
    "Your resolution is not satisfactory. I'm disappointed.",
    "I'm not pleased with your assistance. It's frustrating.",
    "Your service is not helpful. I'm disappointed.",
    "I'm disappointed with your response. It's not satisfactory.",
    "Your service fell short of my expectations. I'm frustrated.",
    "I'm not happy with the outcome. It's disappointing.",
    "Your service is causing inconvenience. Can you fix it?",
    "I'm dissatisfied with your resolution. It's not helpful.",
    "I'm facing problems with your service. Can you assist?",
    "I'm unhappy with your assistance. It's frustrating.",
    "Your support is not satisfactory. I'm not pleased.",
    "I'm frustrated with the inconvenience caused by your service.",
    "I'm disappointed with your resolution. It's not acceptable.",
    "Your service is causing frustration. Can you address it?",
    "I'm dissatisfied with your support. It's not helpful.",
    "I'm not pleased with your assistance. It's frustrating.",
    "Your service is not satisfactory. I'm disappointed.",
    "I'm frustrated with your response. It's not helpful.",
    "I'm not happy with your support. It's disappointing.",
    "Your resolution is not satisfactory. I'm disappointed.",
    "I'm not pleased with your assistance. It's frustrating.",
    "Your service is not helpful. I'm disappointed.",
    "I'm disappointed with your response. It's not satisfactory.",
    "Your service fell short of my expectations. I'm frustrated.",
    "I'm not happy with the outcome. It's disappointing.",
    "Your service is causing inconvenience. Can you fix it?",
    "I'm dissatisfied with your resolution. It's not helpful.",
    "I'm facing problems with your service. Can you assist?",
    "I'm unhappy with your assistance. It's frustrating.",
    "Your support is not satisfactory. I'm not pleased.",
    "I'm frustrated with the inconvenience caused by your service.",
    "I'm disappointed with your resolution. It's not acceptable.",
    "Your service is causing frustration. Can you address it?",
    "I'm dissatisfied with your support. It's not helpful.",
    "I'm not pleased with your assistance. It's frustrating.",
    "Your service is not satisfactory. I'm disappointed.",
    "I'm frustrated with your response. It's not helpful.",
    "I'm not happy with your support. It's disappointing.",
    "Your resolution is not satisfactory. I'm disappointed.",
    "I'm not pleased with your assistance. It's frustrating.",
    "Your service is not helpful. I'm disappointed.",
    "I'm disappointed with your response. It's not satisfactory.",
    "Your service fell short of my expectations. I'm frustrated.",
    "I'm not happy with the outcome. It's disappointing.",
    "Your service is causing inconvenience. Can you fix it?",
    "I'm dissatisfied with your resolution. It's not helpful.",
    "I'm facing problems with your service. Can you assist?",
    "I'm unhappy with your assistance. It's frustrating.",
    "Your support is not satisfactory. I'm not pleased.",
    "I'm frustrated with the inconvenience caused by your service.",
    "I'm disappointed with your resolution. It's not acceptable.",
    "Your service is causing frustration. Can you address it?",
    "I'm dissatisfied with your support. It's not helpful.",
    "I'm not pleased with your assistance. It's frustrating.",
    "Your service is not satisfactory. I'm disappointed.",
    "I'm frustrated with your response. It's not helpful.",
    "I'm not happy with your support. It's disappointing.",
    "Your resolution is not satisfactory. I'm disappointed.",
    "I'm not pleased with your assistance. It's frustrating.",
    "Your service is not helpful. I'm disappointed.",
]

example_data = [{"engineer_response": response, "empathy_label": generate_empathy_label(response)} for response in engineer_responses]

file_path = "empathy_data.csv"

with open(file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    

    writer.writerow(["customer_response", "engineer_response", "empathy_label"])
    

    for customer_response, entry in zip(customer_complaints, example_data):
        writer.writerow([customer_response, entry["engineer_response"], entry["empathy_label"]])

print(f'Data has been saved to {file_path}')

Data has been saved to empathy_data.csv


# Text Preprocessing for NLP

Further preprocesses the text data by tokenizing, removing stopwords, and lemmatizing the words. This step is crucial for effective NLP analysis.

In [15]:
def tokenize_and_lemmatize(text):
    words = word_tokenize(text)

    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
    lemmatized_text = " ".join(lemmatized_words)

    return lemmatized_text

# Read the labeled data we created
test_data = pd.read_csv('empathy_data.csv')  # Replace with the actual path

test_data['processed_customer_response'] = test_data['customer_response'].apply(tokenize_and_lemmatize)

test_data['processed_engineer_response'] = test_data['engineer_response'].apply(tokenize_and_lemmatize)

print(test_data.head())


                                   customer_response  \
0  There has been a mistake in your service. I'm ...   
1  I'm not happy with your response. It wasn't he...   
2        Your service is terrible. I'm disappointed.   
3  I'm frustrated with the poor service. Can you ...   
4  Your assistance didn't address my concerns. I'...   

                                   engineer_response  empathy_label  \
0  I understand your frustration. Let's work on a...              6   
1  I apologize for the poor service. We'll invest...              3   
2       You're welcome! I'm glad I could assist you.             10   
3  I'm sorry for the inconvenience. Let me see wh...              3   
4  I'm sorry to hear that. Can you provide more d...              6   

                     processed_customer_response  \
0               mistake service . 'm satisfied .   
1              'm happy response . n't helpful .   
2           service terrible . 'm disappointed .   
3             'm frustrated 

# Training and Testing Machine Learning Models

Includes code for training and evaluating machine learning models (Logistic Regression, Random Forest) on the empathy scoring task. The cell also covers model hyperparameter tuning and performance evaluation.

In [16]:
X = [" ".join([row['processed_customer_response'], row['processed_engineer_response']]) for _, row in test_data.iterrows()]
y = test_data['empathy_label']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

model = LogisticRegression()
y_train = y_train.astype(int)
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


X_original = [" ".join([row['customer_response'], row['engineer_response']]) for _, row in original_data.iterrows()]
X_original_vec = vectorizer.transform(X_original)


y_original_pred = model.predict(X_original_vec)
for _, row in original_data.iterrows():
    print("Sentence: ",row['engineer_response'],"\n")
    print("Empathy Score: ",y_original_pred[_])
    if(y_original_pred[_]>=7):
        print("Status: PASS\n")
    else:
        print("Status: FAIL","\n","Comment: Engineer needs to be more empathetic\n")
    
    

Accuracy: 0.7
Sentence:  Hi Roberto, We have de-staged the stores with lastpos and provided menu maker access. Store UUID : 7dad6eeb-4746 A46d97e2-e99 A26f3b8a-4ffc 02 Efb224ca-04 If you feel your request was not addressed/resolved properly, you can reopen the same case within 3 business working days and/or request escalation. If you have a new issue, please raise a new case. Regards, L2 UET Support 

Empathy Score:  6
Status: FAIL 
 Comment: Engineer needs to be more empathetic

Sentence:  Hi Stefan, Please follow the below documents to update the modifiers in the menu payload. https://develo https:/develo Store UUID: 7140faa If you feel the issue is not addressed/resolved properly you can reopen the same case within 3 business working days or If you have a new issue please raise a new case. Thanks, L2 UET Support 

Empathy Score:  6
Status: FAIL 
 Comment: Engineer needs to be more empathetic

Sentence:  Hi Pankaj, Please try understanding the situation. All the menu changes need to 

# Hyperparameter tuning for Random Classifier 

Hyperparameter tuning applied for Random Classifier to see if accuracy can be approved.

*Apparently giving same accuracy.

In [17]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

model = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(model, param_grid, cv=10, scoring='accuracy')

grid_search.fit(X_train_vec, y_train)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test_vec)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Best Hyperparameters:", grid_search.best_params_)


C:\Users\SNEHA\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


Accuracy: 0.7
Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 50}


# Random Classifier

Using this model as this is apt for a data set so small.Also able to achieve 70% accuracy due to inadequate length of dataset.Labeled data created manually for training model.

In [18]:
# Train a Random Forest classifier
# Use TF-IDF Vectorization for tokenization and vectorization
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

model1 = RandomForestClassifier(n_estimators=100, random_state=42)
model1.fit(X_train_vec, y_train)


y_pred = model1.predict(X_test_vec)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
y_original_pred = model1.predict(X_original_vec)
for _, row in original_data.iterrows():
    print("Sentence: ",row['engineer_response'])
    print("Empathy Score: ",y_original_pred[_])
    if(y_original_pred[_]>=7):
        status="Pass"
        comment="Nil"
        print("Status: PASS\n")
    else:
        status="Fail"
        comment="Engineer needs to be more empathetic"
        print("Status: FAIL","Comment: Engineer needs to be more empathetic\n")
    result_data.append({"ticket_id": row['ticket_id'], "parameter": "Empathy","score":y_original_pred[_],"status":status,"comment":comment})
    

Accuracy: 0.7
Sentence:  Hi Roberto, We have de-staged the stores with lastpos and provided menu maker access. Store UUID : 7dad6eeb-4746 A46d97e2-e99 A26f3b8a-4ffc 02 Efb224ca-04 If you feel your request was not addressed/resolved properly, you can reopen the same case within 3 business working days and/or request escalation. If you have a new issue, please raise a new case. Regards, L2 UET Support
Empathy Score:  6
Status: FAIL Comment: Engineer needs to be more empathetic

Sentence:  Hi Stefan, Please follow the below documents to update the modifiers in the menu payload. https://develo https:/develo Store UUID: 7140faa If you feel the issue is not addressed/resolved properly you can reopen the same case within 3 business working days or If you have a new issue please raise a new case. Thanks, L2 UET Support
Empathy Score:  6
Status: FAIL Comment: Engineer needs to be more empathetic

Sentence:  Hi Pankaj, Please try understanding the situation. All the menu changes need to be done 

# Result 

Result set having ticket id as column for reference and its respective SCORE, STATUS and COMMENT for each parameter GRAMMER and EMPATHY. Result set also saved as csv in your dir.

In [19]:
result_df = pd.DataFrame(result_data)
result_df.to_excel('output.xlsx', index=False)
print(result_df)

  ticket_id parameter  score status  \
0     34333   grammar      6   Fail   
1       338   grammar     10   Pass   
2        33   grammar      6   Fail   
3       337   grammar      1   Fail   
4     34333   Empathy      6   Fail   
5       338   Empathy      6   Fail   
6        33   Empathy      6   Fail   
7       337   Empathy      6   Fail   

                                             comment  
0  Possible spelling mistake found. (Suggested wo...  
1                       No grammatical errors found.  
2  Possible spelling mistake found. (Suggested wo...  
3  Possible spelling mistake found. (Suggested wo...  
4               Engineer needs to be more empathetic  
5               Engineer needs to be more empathetic  
6               Engineer needs to be more empathetic  
7               Engineer needs to be more empathetic  
